In [8]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json

# Paquetes de consulta a traves de Socrata

from sodapy import Socrata
identificador_columnas=pd.read_csv('datos/identificador_columnas.csv')
valor_columnas={'SECOP I':'API_DA_S_I','SECOP II':'API_DA_S_II','TVEC':'API_DA_S_TVEC'}
identificador_columnas=identificador_columnas[['Unificado','API_DA_S_I', 'API_DA_S_II']]
identificador_columnas=identificador_columnas.dropna()
identificador_columnas=identificador_columnas.set_index('Unificado')
identificador_columnas.drop('Fuente',axis=0,inplace=True)
identificador_datos={'SECOP I':'f789-7hwg','SECOP II':'jbjy-vk9h','TVEC':'rgxm-mmea'}
valor_columnas={'SECOP I':'API_DA_S_I','SECOP II':'API_DA_S_II','TVEC':'API_DA_S_TVEC'}
### Definición de cliente para Socrata

cliente=Socrata('www.datos.gov.co',None,timeout=1000000)

In [9]:
with open('datos/dptos.json') as json_file:
    dptos = json.load(json_file)

In [4]:
def consulta_departamento(departamento):
    results_detalle=[]
    for i in ['SECOP I','SECOP II']:
        queries_list=[]
        results_detalle_dpto=[]
        for j in dptos[departamento]:
            query_secop_tot=""" Select
                {0} as id_contrato,
                {1} as id_proceso,
                {2} as valor_contrato,
                {3} as entidad,
                {4} as departamento_entidad,
                {5} as orden,
                {6} as modalidad,
                {7} as tipo_contrato,
                {8} as codigo_unspsc,
                {9} as objeto_contrato,
                {10} as descripcion_proceso,
                {11} as nombre_proveedor,
                {12} as tipo_documento_proveedor,
                {13} as numero_documento_proveedor,
                {14} as fecha_inicio,
                {15} as fecha_fin,
                {16} as fecha_firma,
                {17} as url

            where
                {4}='{18}'
            limit
                10000000
            """.format(identificador_columnas[valor_columnas[i]]['ID Contrato'],identificador_columnas[valor_columnas[i]]['ID Proceso'],
            identificador_columnas[valor_columnas[i]]['Valor del contrato'],
            identificador_columnas[valor_columnas[i]]['Entidad'],identificador_columnas[valor_columnas[i]]['Departamento Entidad'],
            identificador_columnas[valor_columnas[i]]['Orden'],identificador_columnas[valor_columnas[i]]['Modalidad'],
            identificador_columnas[valor_columnas[i]]['Tipo de contrato'],identificador_columnas[valor_columnas[i]]['UNSPSC'],
            identificador_columnas[valor_columnas[i]]['Objeto Contrato'],identificador_columnas[valor_columnas[i]]['Descripción proceso'],
            identificador_columnas[valor_columnas[i]]['Nombre Proveedor'],identificador_columnas[valor_columnas[i]]['Tipo de documento proveedor'],
            identificador_columnas[valor_columnas[i]]['Documento Proveedor'],identificador_columnas[valor_columnas[i]]['Inicio de contrato'],
            identificador_columnas[valor_columnas[i]]['Fin de contrato'],identificador_columnas[valor_columnas[i]]['Fecha de Firma'],
            identificador_columnas[valor_columnas[i]]['URL'],j)
            queries_list.append(query_secop_tot)
            df_temp=pd.DataFrame(cliente.get(identificador_datos[i], query=query_secop_tot))
            df_temp['fuente']=i
            if df_temp.shape[0]>0:
                results_detalle_dpto.append(df_temp)
        if len(results_detalle_dpto)>0:
            results_detalle.append(pd.concat(results_detalle_dpto))
    results_detalle=pd.concat(results_detalle)
    results_detalle.reset_index(inplace=True)
    results_detalle.drop('index',axis=1,inplace=True)
    results_detalle['valor_contrato']=results_detalle['valor_contrato'].astype(float)
    results_detalle['departamento_entidad']=results_detalle['departamento_entidad'].replace('Distrito Capital de Bogotá','Bogotá DC')\
    .replace('San Andrés Providencia y Santa Catalina','San Andrés, Providencia y Santa Catalina').replace('Norte De Santander','Norte de Santander')
      
    return results_detalle

In [64]:
def consulta_mae(departamento,entidad,fecha_firma_inicio,fecha_firma_fin,valor_min,valor_max,cod_unspsc):
    results_detalle=[]
    for i in ['SECOP I','SECOP II']:
        queries_list=[]
        results_detalle_dpto=[]
        for j in dptos[departamento]:
            query_secop_tot=""" Select
                {0} as id_contrato,
                {1} as id_proceso,
                {2} as valor_contrato,
                {3} as entidad,
                {4} as departamento_entidad,
                {5} as orden,
                {6} as modalidad,
                {7} as tipo_contrato,
                {8} as codigo_unspsc,
                {9} as objeto_contrato,
                {10} as descripcion_proceso,
                {11} as nombre_proveedor,
                {12} as tipo_documento_proveedor,
                {13} as numero_documento_proveedor,
                {14} as fecha_inicio,
                {15} as fecha_fin,
                {16} as fecha_firma,
                {17} as url

            where
                ({4}='{18}') and
                ({3}='{19}') and
                ({16} between '{20}' and '{21}') and
                ({2} between {22} and {23}) and
                {8} like '%{24}%'
            limit
                10000000
            """.format(identificador_columnas[valor_columnas[i]]['ID Contrato'],identificador_columnas[valor_columnas[i]]['ID Proceso'],
            identificador_columnas[valor_columnas[i]]['Valor del contrato'],
            identificador_columnas[valor_columnas[i]]['Entidad'],identificador_columnas[valor_columnas[i]]['Departamento Entidad'],
            identificador_columnas[valor_columnas[i]]['Orden'],identificador_columnas[valor_columnas[i]]['Modalidad'],
            identificador_columnas[valor_columnas[i]]['Tipo de contrato'],identificador_columnas[valor_columnas[i]]['UNSPSC'],
            identificador_columnas[valor_columnas[i]]['Objeto Contrato'],identificador_columnas[valor_columnas[i]]['Descripción proceso'],
            identificador_columnas[valor_columnas[i]]['Nombre Proveedor'],identificador_columnas[valor_columnas[i]]['Tipo de documento proveedor'],
            identificador_columnas[valor_columnas[i]]['Documento Proveedor'],identificador_columnas[valor_columnas[i]]['Inicio de contrato'],
            identificador_columnas[valor_columnas[i]]['Fin de contrato'],identificador_columnas[valor_columnas[i]]['Fecha de Firma'],
            identificador_columnas[valor_columnas[i]]['URL'],j,entidad,fecha_firma_inicio,fecha_firma_fin,valor_min,valor_max,cod_unspsc)
            queries_list.append(query_secop_tot)
            df_temp=pd.DataFrame(cliente.get(identificador_datos[i], query=query_secop_tot))
            df_temp['fuente']=i
            if df_temp.shape[0]>0:
                results_detalle_dpto.append(df_temp)
        if len(results_detalle_dpto)>0:
            results_detalle.append(pd.concat(results_detalle_dpto))
    results_detalle=pd.concat(results_detalle)
    results_detalle.reset_index(inplace=True)
    results_detalle.drop('index',axis=1,inplace=True)
    results_detalle['valor_contrato']=results_detalle['valor_contrato'].astype(float)
    results_detalle['departamento_entidad']=results_detalle['departamento_entidad'].replace('Distrito Capital de Bogotá','Bogotá DC')\
    .replace('San Andrés Providencia y Santa Catalina','San Andrés, Providencia y Santa Catalina').replace('Norte De Santander','Norte de Santander')
      
    return results_detalle


In [66]:
df=consulta_mae('Bogotá DC','SERVICIO NACIONAL DE APRENDIZAJE SENA','2018-01-01','2019-12-31','0','100000000000','801116')
df

,id_contrato,id_proceso,valor_contrato,entidad,departamento_entidad,orden,modalidad,tipo_contrato,codigo_unspsc,objeto_contrato,descripcion_proceso,nombre_proveedor,tipo_documento_proveedor,numero_documento_proveedor,fecha_inicio,fecha_fin,fecha_firma,url,fuente
0,12,19-12-9464395-8613240,21502371.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",PRESTACIÓN DE LOS SERVICIOS PERSONALES DE CARÁ...,ARGIRO DE JESUS VERGARA MUÑOZ,Nit de Persona Natural,98571173,2019-01-18T00:00:00.000,2019-12-24T00:00:00.000,2019-01-17T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
1,RV1332018,18-12-8692588-7903196,3600000.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",Prestar los servicios personales como Instruct...,SILVIA CAROLA OROZCO PEÑA,Cédula de Ciudadanía,36289485,2018-11-16T00:00:00.000,2018-12-16T00:00:00.000,2018-11-16T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
2,790,18-12-7847857-7123988,21456000.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",Prestar los servicios profesionales de carácte...,SERGIO EDMUNDO BARAJAS ACEVEDO,Cédula de Ciudadanía,13926481,2018-02-01T00:00:00.000,2018-10-02T00:00:00.000,2018-01-24T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
3,52704,19-12-9556305-8698874,18000000.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",Prestación de servicios personales de carácte...,ANA MARIA VALENCIA SUAREZ,Cédula de Ciudadanía,31985511,2019-06-10T00:00:00.000,2019-12-10T00:00:00.000,2019-06-06T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
4,155,18-12-8787434-7988022,11345920.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",Prestar los servicios profesionales como instr...,LINA ROSA VERA ANGARITA,Cédula de Ciudadanía,68247836,2018-10-09T00:00:00.000,2018-12-13T00:00:00.000,2018-10-09T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36879,198,19-12-8987692-8175427,37800000.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",Prestar servicios personales de carácter tempo...,AZUCENA MARIA GARCIA CRUZ,Cédula de Ciudadanía,1088040292,2019-02-04T00:00:00.000,2019-12-16T00:00:00.000,2019-02-04T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
36880,4912,19-12-9689261-8819779,19257457.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",Prestación de servicios de carácter temporal p...,LUIS ALEJANDRO CEDIEL TELLO,Cédula de Ciudadanía,19452407,2019-07-10T00:00:00.000,2019-12-14T00:00:00.000,2019-07-10T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
36881,867,19-12-9200360-8371892,35693643.0,SERVICIO NACIONAL DE APRENDIZAJE SENA,Bogotá DC,NACIONAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",CONTRATAR LA PRESTACION DE SERVICIOS PROFESION...,FREDDY CAMACHO GARCIA,Cédula de Ciudadanía,1110503474,2019-02-26T00:00:00.000,2019-12-25T00:00:00.000,2019-02-25T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta

In [5]:
def consulta_datos_departamento(departamento,año):
    df_consulta=pd.read_csv('datos/contratos_entidad.csv')
    df_consulta=df_consulta[df_consulta['departamento_entidad']==departamento]
    df_consulta=df_consulta[df_consulta['año']==año]
    df_consulta['suma_valor_contrato']=df_consulta['suma_valor_contrato'].astype(float)
    df_consulta['cantidad']=df_consulta['cantidad'].astype(int)
    df_consulta['departamento_entidad']=df_consulta['departamento_entidad'].replace('Distrito Capital de Bogotá','Bogotá DC')\
    .replace('San Andrés Providencia y Santa Catalina','San Andrés, Providencia y Santa Catalina').replace('Norte De Santander','Norte de Santander')
    return df_consulta

In [55]:
consulta_departamento('San Andrés, Providencia y Santa Catalina')

,id_contrato,id_proceso,valor_contrato,entidad,departamento_entidad,orden,modalidad,tipo_contrato,codigo_unspsc,objeto_contrato,descripcion_proceso,nombre_proveedor,tipo_documento_proveedor,numero_documento_proveedor,fecha_inicio,fecha_fin,fecha_firma,url,fuente
0,06818,18-4-8778420-7979584,9910000.0,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA INSTI...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Régimen Especial,Obra,721033,"Servicios de Edificación, Construcción de Inst...",SERVICIOS MANTENIMIENTO A TODO COSTO DE INFRAE...,JONATHAN MANUEL PEREZ MERCADO,Cédula de Ciudadanía,1123626969,2018-12-13T00:00:00.000,2018-12-21T00:00:00.000,2018-12-13T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
1,CD7302021,21-12-12139046-11243285,8060000.0,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA ALCAL...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,941215,Organizaciones y Clubes,Prestación de servicio de apoyo a la gestión c...,LAUREANA TAYLOR SMITH,Cédula de Ciudadanía,1120980309,2021-02-02T00:00:00.000,2021-07-02T00:00:00.000,2021-02-02T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
2,CD N 020 DE 2016,16-12-4966870-4553694,11737440.0,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA ALCAL...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",EL CONTRATISTA SE OBLIGA PARA CON EL MUNICIPIO...,FRANCISCO JOSE HUFFINGTON BRITTON,Cédula de Ciudadanía,8724739,2016-02-01T00:00:00.000,2016-08-01T00:00:00.000,2016-02-01T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
3,No definido,16-12-5968329,NaN,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA ALCAL...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",SERVICIOS TEMPORALES PERSONALES PARA LABORES D...,No Definido,No Definido,No Definido,NaN,NaN,NaN,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
4,541 DE 2015,15-12-3736744-3494849,28922634.0,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA GOBER...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,N/D,"Servicios de Gestion, Servicios Profesionales ...",Prestar sus servicios profesionales como Biólo...,CRISTINA ANDREA ZAPATA MASPOLI,Cédula de Ciudadanía,40992274,2015-03-27T00:00:00.000,2015-12-27T00:00:00.000,2015-03-27T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49143,CO1.PCCNTR.1828972,CO1.BDOS.1441050,10055535.0,GOBERNACIÓN DEL DEPARTAMENTO ARCHIPIELAGO DE S...,"San Andrés, Providencia y Santa Catalina",Territorial,Contratación directa,Prestación de servicios,V1.80111600,Prestación de servicios profesionales como adm...,Prestación de servicios profesionales como adm...,DEVI ESTIGUAR ROMERO HERAZO,Cédula de Ciudadanía,1123627320,2020-09-22T00:00:00.000,2020-12-30T00:00:00.000,2020-09-14T16:09:53.000,{'url': 'https://community.secop.gov.co/Public...,SECOP II
49144,CO1.PCCNTR.3542311,CO1.BDOS.2800444,10136565.0,GOBERNACIÓN DEL DEPARTAMENTO ARCHIPIELAGO DE S...,"San Andrés, Providencia y Santa Catalina",Territorial,Contratación directa,Prestación de servicios,V1.80111600,Prestacion de servicios como bachiller academi...,Prestacion de servicios como bachiller academi...,Kelineth Coleth Kelly right,Cédula de Ciudadanía,1007416615,2022-02-02T00:00:00.000,2022-07-01T00:00:00.000,2022-02-01T18:02:30.000,{'url': 'https://community.secop.gov.co/Public...,SECOP II
49145,CO1.PCCNTR.2335241,CO1.BDOS.1833624,34065027.0,GOBERNACIÓN DEL DEPARTAMENTO ARCHIPIELAGO DE S...,"San Andrés, Providencia y Santa Catalina",Territorial,Contratación directa,Prestación de servicios,V1.80111600,PRESTACION DE SERVICIOS PROFESIONALE

In [34]:
consulta_mae('Antioquia','ANTIOQUIA ALCALDÍA MUNICIPIO DE ZARAGOZA','2018','2019-01-01','2019-12-31','0','1000000000')

ValueError: No objects to concatenate